In [14]:
for package in search('http'):
    print(package)

Package(name='http', version='0.02', released='2012-02-12T01:53:36+0000', description='HTTP library for Python')
Package(name='http1', version='1.0.2', released='2020-05-10T17:09:42+0000', description='http1 is an API to perform HTTP requests in a single call')
Package(name='http3', version='0.6.7', released='2019-07-08T14:58:54+0000', description='The next generation HTTP client.')
Package(name='opensitua-http', version='0.0.72', released='2021-06-29T14:06:00+0000', description='A core functions package')
Package(name='os-http', version='0.1.0', released='2016-04-11T06:20:41+0000', description='Simple HTTP CLI for OpenStack')
Package(name='pcap2http', version='1.1', released='2016-05-29T18:08:55+0000', description='Parse pcap file with python and watch sites in a browser or a file system')
Package(name='progressed-http', version='1.0.0', released='2017-10-08T08:56:39+0000', description='Personal http req/resp handler with origin progress bar support')
Package(name='pulumi-http', versi

In [ ]:
def search(
    query: str, opts: Union[dict, Namespace] = {}
) -> Generator[Package, None, None]:
    """Search for packages matching the query

    Yields:
        Package: package object
    """
    snippets = []
    s = requests.Session()
    for page in range(1, config.page_size + 1):
        params = {"q": query, "page": page}
        r = s.get(config.api_url, params=params)
        soup = BeautifulSoup(r.text, "html.parser")
        snippets += soup.select('a[class*="package-snippet"]')
    if "sort" in opts:
        if opts.sort == "name":
            snippets = sorted(
                snippets,
                key=lambda s: s.select_one('span[class*="package-snippet__name"]').text.strip(),
            )
            print(snippets)
            snippets = sorted(
                snippets,
                key=lambda s: parse_version(
                    s.select_one('span[class*="package-snippet__version"]').text.strip()
                ),
            )
            print(snippets)
        elif opts.sort == "version":
            from pkg_resources import parse_version

            snippets = sorted(
                snippets,
                key=lambda s: parse_version(
                    s.select_one('span[class*="package-snippet__version"]').text.strip()
                ),
            )
        elif opts.sort == "released":
            snippets = sorted(
                snippets,
                key=lambda s: s.select_one('span[class*="package-snippet__created"]').find(
                    "time"
                )["datetime"],
            )
    for snippet in snippets:
        link = urljoin(config.api_url, snippet.get("href"))
        package = re.sub(
            r"\s+", " ", snippet.select_one('span[class*="package-snippet__name"]').text.strip()
        )

        response = s.get(link)
        package_page = BeautifulSoup(response.text, "html.parser")
        version_element = package_page.select_one('h1.package-header__name')
        version = version_element.text.split()[-1] if version_element else "Unknown"

        # version = re.sub(
        #     r"\s+",
        #     " ",
        #     snippet.select_one('span[class*="package-snippet__version"]').text.strip(),
        # )
        released = re.sub(
            r"\s+",
            " ",

            snippet.select_one('span[class*="package-snippet__created"]').find("time")[
                "datetime"
            ],
        )
        description = re.sub(
            r"\s+",
            " ",
            snippet.select_one('p[class*="package-snippet__description"]').text.strip(),
        )
        yield Package(package, version, released, description, link)

In [ ]:
import re
from argparse import Namespace
from dataclasses import InitVar, dataclass
from datetime import datetime
from typing import Generator, Union
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup


class Config:
    """Configuration class"""

    api_url: str = "https://pypi.org/search/"
    page_size: int = 2
    sort_by: str = "name"
    date_format: str = "%d-%-m-%Y"
    link_defualt_format: str = "https://pypi.org/project/{package.name}"


config = Config()


@dataclass
class Package:
    """Package class"""

    name: str
    version: str
    released: str
    description: str
    link: InitVar[str] = None

    def __post_init__(self, link: str = None):
        self.link = link or config.link_defualt_format.format(package=self)
        self.released_date = datetime.strptime(
            self.released, "%Y-%m-%dT%H:%M:%S%z"
        )

    def released_date_str(self, date_format: str = config.date_format) -> str:
        """Return the released date as a string formatted
        according to date_formate ou Config.date_format (default)

        Returns:
            str: Formatted date string
        """
        return self.released_date.strftime(date_format)
